# Instantiate ICTV OLS Client

In [30]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json

from ictv_ols import ICTVOLSClient
client = ICTVOLSClient('https://www.ebi.ac.uk/ols4/api/v2/ontologies/ictv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get taxa by release

In [25]:

msl_7_viruses = client.get_all_taxa_by_release('MSL7')
print(f"Number of viruses in MSL7: {len(msl_7_viruses)}")

df = pd.DataFrame(msl_7_viruses)[['msl', 'ictv_id', 'label']]
print(df.head(10).to_markdown())



Number of viruses in MSL7: 1221
|    | msl   | ictv_id      | label                      |
|---:|:------|:-------------|:---------------------------|
|  0 | MSL7  | ICTV19790785 | Aus MK 6357 virus          |
|  1 | MSL7  | ICTV19710002 | Papovaviridae              |
|  2 | MSL7  | ICTV19710003 | Picornaviridae             |
|  3 | MSL7  | ICTV19710008 | Papillomavirus             |
|  4 | MSL7  | ICTV19710009 | Polyomavirus               |
|  5 | MSL7  | ICTV19710010 | Calicivirus                |
|  6 | MSL7  | ICTV19710011 | Enterovirus                |
|  7 | MSL7  | ICTV19710012 | Rhinovirus                 |
|  8 | MSL7  | ICTV19710013 | Mastadenovirus             |
|  9 | MSL7  | ICTV19710014 | Alfalfa mosaic virus group |


# Get specific taxon by release

In [31]:

picornaviridae = client.get_taxon_by_release('MSL7', 'Picornaviridae')
print(json.dumps(picornaviridae, indent=2))


{
  "msl": "MSL7",
  "ictv_id": "ICTV19710003",
  "label": "Picornaviridae",
  "obsolescence_reason": null,
  "taxonomic_parents": [
    "http://ictv.global/id/MSL7/ICTV19710000"
  ],
  "historical_parents": [
    "http://ictv.global/id/MSL6/ICTV19710003"
  ],
  "replacements": [
    "http://ictv.global/id/MSL8/ICTV19710003"
  ]
}


### And get its taxonomic parents

In [32]:

parents = client.get_taxonomic_parents('MSL7', 'Cardiovirus')

df = pd.DataFrame(parents)[['ictv_id', 'label']]
print(df.head(10))



        ictv_id           label
0  ICTV19710003  Picornaviridae


# Getting the history of a taxon

### What happened to Bovine adenovirus from MSL6?

In [20]:
taxon = client.get_taxon_by_release('MSL6', 'Bovine adenovirus')
replacements = client.get_replacements('MSL6', 'Bovine adenovirus')

print(f"Obsolescence reason: {taxon['obsolescence_reason']}")
print(f"Replacements:")

print(pd.DataFrame(replacements)[['msl', 'ictv_id', 'label']].to_markdown())



Obsolescence reason: SPLIT
Replacements:
|    | msl   | ictv_id      | label               |
|---:|:------|:-------------|:--------------------|
|  0 | MSL7  | ICTV19810183 | Bovine adenovirus 1 |
|  1 | MSL7  | ICTV19810184 | Bovine adenovirus 2 |
|  2 | MSL7  | ICTV19810185 | Bovine adenovirus 3 |
|  3 | MSL7  | ICTV19810186 | Bovine adenovirus 4 |
|  4 | MSL7  | ICTV19810187 | Bovine adenovirus 5 |
|  5 | MSL7  | ICTV19810188 | Bovine adenovirus 6 |
|  6 | MSL7  | ICTV19810189 | Bovine adenovirus 7 |
|  7 | MSL7  | ICTV19810190 | Bovine adenovirus 8 |
|  8 | MSL7  | ICTV19810191 | Bovine adenovirus 9 |


### Where did Poliovirus in MSL18 come from?

In [22]:
parents = client.get_historical_parents('MSL18', 'Poliovirus')

print(pd.DataFrame(parents)[['msl', 'ictv_id', 'label', 'obsolescence_reason']].to_markdown())

|    | msl   | ictv_id      | label              | obsolescence_reason   |
|---:|:------|:-------------|:-------------------|:----------------------|
|  0 | MSL17 | ICTV19710068 | Human poliovirus 1 | MERGED                |
|  1 | MSL17 | ICTV19790556 | Human poliovirus 2 | MERGED                |
|  2 | MSL17 | ICTV19790557 | Human poliovirus 3 | MERGED                |


### Where did Alphacytorhabdovirus in MSL40 come from?

In [23]:
parents = client.get_historical_parents('MSL40', 'Alphacytorhabdovirus')

print(pd.DataFrame(parents)[['msl', 'ictv_id', 'label', 'obsolescence_reason']].to_markdown())


|    | msl   | ictv_id      | label           | obsolescence_reason   |
|---:|:------|:-------------|:----------------|:----------------------|
|  0 | MSL39 | ICTV19820133 | Cytorhabdovirus | SPLIT                 |


#### ... and what else was Cytorhabdovirus split into?

In [24]:
replacements = client.get_replacements('MSL39', 'Cytorhabdovirus')

print(pd.DataFrame(replacements)[['msl', 'ictv_id', 'label']].to_markdown())

|    | msl   | ictv_id       | label                |
|---:|:------|:--------------|:---------------------|
|  0 | MSL40 | ICTV202419770 | Alphacytorhabdovirus |
|  1 | MSL40 | ICTV202419771 | Betacytorhabdovirus  |
|  2 | MSL40 | ICTV202419772 | Gammacytorhabdovirus |


# What happened to MSL38 Stockinghallvirus?

In [41]:
replacements = client.get_replacements('MSL38', 'Stockinghallvirus')

print(replacements)



[]
